In [31]:
import os
import joblib
import pickle as pkl

In [32]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib

#Load data
boston = datasets.load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df['MEDV'] = boston.target 

#Split Model
X = df.drop(['MEDV'], axis = 1) 
y = df['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

#Model Creation
lm = LinearRegression()
lm.fit(X_train,y_train)


with open('model.joblib', 'wb') as f:
    joblib.dump(lm,f)


with open('model.joblib', 'rb') as f:
    predictor = joblib.load(f)

print("Testing following input: ")
print(X_test[0:1])
sampInput = [[0.09178, 0.0, 4.05, 0.0, 0.51, 6.416, 84.1, 2.6463, 5.0, 296.0, 16.6, 395.5, 9.04]]
print(type(sampInput))
print(predictor.predict(sampInput))

Testing following input: 
        CRIM   ZN  INDUS  CHAS   NOX     RM   AGE     DIS  RAD    TAX  \
173  0.09178  0.0   4.05   0.0  0.51  6.416  84.1  2.6463  5.0  296.0   

     PTRATIO      B  LSTAT  
173     16.6  395.5   9.04  
<class 'list'>
[28.99672362]


/Users/sharhad/python_versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Californi

In [15]:
text = 'day fire starter people fire light 🔥 minister gospel commitment spreading god grace power'
# output = 8

In [16]:
def load_pkl_model(path):
    return pkl.load(open(path, 'rb'))

def convert_pkl_joblib(model, language = 'en'):
    with open(f'../models/model_{language}.joblib', 'wb') as file:
        joblib.dump(model, file)
    print('Model converted')
    
def load_joblib_model(path):
    with open(path, 'rb') as file:
        model = joblib.load(file)
    return model

In [17]:
model_pkl = load_pkl_model('../models/model.pkl')
print('pkl model prediction:', model_pkl.predict([text])[0])
convert_pkl_joblib(model_pkl)
model_joblib = load_joblib_model('../models/model_en.joblib')
print('joblib model prediction:', model_joblib.predict([text])[0])

/Users/sharhad/python_versions/3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.2.0 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/sharhad/python_versions/3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.0 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/sharhad/python_versions/3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.0 when using version 1.1.3. T

pkl model prediction: 8
Model converted
joblib model prediction: 8


In [6]:
"""
Deserialize fitted model
"""
def model_fn(model_dir, language = 'en'):
    model = joblib.load(os.path.join(model_dir, f'model_{language}.joblib'))
    return model

"""
input_fn
    request_body: The body of the request sent to the model.
    request_content_type: (string) specifies the format/variable type of the request
"""
def input_fn(request_body):
    if (type(request_body) == str):
        return [request_body]
    else:
        raise ValueError('This model only supports string inputs')

"""
predict_fn
    input_data: returned array from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    return model.predict(input_data)[0] #could be bug here

"""
output_fn
    prediction: the returned value from predict_fn above
    content_type: the content type the endpoint expects to be returned. Ex: JSON, string

"""
def output_fn(prediction):
    return int(prediction[0]) #could be bug here

In [25]:
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

In [26]:
def bundle(language = 'en'):
    bashCommand = 'tar -cvpzf model_en.tar.gz model_en.joblib inference.py'
    process = subprocess.Popen(bashCommand.split(), stdout = subprocess.PIPE)
    output, error = process.communicate()
    print('Output:', output)  
    if error:
        print('Error:', error)

def upload(language = 'en', default_bucket = 'ts-contextual-web-api'):
    boto_session = boto3.session.Session()
    s3 = boto_session.resource('s3')
    boto_session = boto3.session.Session()
    response = s3.meta.client.upload_file('model_en.tar.gz', default_bucket, 'model_en.tar.gz')
    print('Response:', response)

In [27]:
bundle()
upload()
s3_client = boto3.client('s3')
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])

a model_en.joblib
a inference.py


Output: b''
Response: None
dev-bluebox
ops-vault-eu-aws-state
ops-vault-prd
ops-vault-stg
sc-eu-aws-state
tf-poc-aws-state
tf-poceu-aws-state
ts-contextual-web-api
ts-fill-in-blanks
ts-podcast-iq
ts-ryan
ts-temp-bucket
ts-transcription
ts-veeam-storage
ts-whisper
tsadobe
tsdigiseg
tsivt
tsliveramp
tsppiq
tstruoptik


In [28]:
default_bucket = 'ts-contextual-web-api'
client = boto3.client(service_name = 'sagemaker')
runtime = boto3.client(service_name = 'sagemaker-runtime')
model_artifacts = f's3://{default_bucket}/model_en.tar.gz'

image_uri = sagemaker.image_uris.retrieve(
    framework = 'sklearn',
    region = 'us-east-1',
    version = '0.23-1',
    py_version = 'py3',
    instance_type = 'ml.m5.2xlarge'
)

model_name = 'sklearn-test' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Model name:', model_name)
create_model_response = client.create_model(
    ModelName = model_name,
    Containers = [
        {
            'Image': image_uri,
            'Mode': 'SingleModel',
            'ModelDataUrl': model_artifacts,
            'Environment': {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn = 'arn:aws:iam::052845409385:role/AmazonSageMaker-ExecutionRole'
)
print('Model Arn: ' + create_model_response['ModelArn'])

Model name: sklearn-test2023-05-05-16-55-39
Model Arn: arn:aws:sagemaker:us-east-1:052845409385:model/sklearn-test2023-05-05-16-55-39


In [29]:
sklearn_epc_name = 'sklearn-epc' + strftime('%Y-%m-%d-%H-%M-%S', gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = sklearn_epc_name,
    ProductionVariants = [
        {
            'VariantName': 'sklearnvariant',
            'ModelName': model_name,
            'InstanceType': 'ml.c5.4xlarge',
            'InitialInstanceCount': 1
        },
    ],
)
print('Endpoint Configuration Arn: ' + endpoint_config_response['EndpointConfigArn'])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:052845409385:endpoint-config/sklearn-epc2023-05-05-16-55-44


In [30]:
endpoint_name = 'sklearn-local-ep' + strftime('%Y-%m-%d-%H-%M-%S', gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName = endpoint_name,
    EndpointConfigName = sklearn_epc_name,
)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])


#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName = endpoint_name)
while describe_endpoint_response['EndpointStatus'] == 'Creating':
    describe_endpoint_response = client.describe_endpoint(EndpointName = endpoint_name)
    print(describe_endpoint_response['EndpointStatus'])
    time.sleep(150)
print(describe_endpoint_response)

Endpoint Arn: arn:aws:sagemaker:us-east-1:052845409385:endpoint/sklearn-local-ep2023-05-05-16-55-49
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Failed
{'EndpointName': 'sklearn-local-ep2023-05-05-16-55-49', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:052845409385:endpoint/sklearn-local-ep2023-05-05-16-55-49', 'EndpointConfigName': 'sklearn-epc2023-05-05-16-55-44', 'EndpointStatus': 'Failed', 'FailureReason': 'The primary container for production variant sklearnvariant did not pass the ping health check. Please check CloudWatch logs for this endpoint.', 'CreationTime': datetime.datetime(2023, 5, 5, 12, 55, 49, 815000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2023, 5, 5, 13, 19, 20, 247000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '0ba012e8-711f-48f0-b51d-805b621bac1e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0ba012e8-711f-48f0-b51d-805b621bac1e', 'content-type': 'application/x-amz-json-1.1', 'c

In [ ]:
runtime_client = boto3.client('sagemaker-runtime')
content_type = str
# request_body = {'Input': [[0.09178, 0.0, 4.05, 0.0, 0.51, 6.416, 84.1, 2.6463, 5.0, 296.0, 16.6, 395.5, 9.04]]}
# data = json.loads(json.dumps(request_body))
payload = json.dumps(text)
# endpoint_name = 'enter endpoint name here'

response = runtime_client.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = content_type,
    Body = payload)
result = json.loads(response['Body'].read().decode())['Output']